# Decision tree

설정한 여러가지 조건을 토대로 데이터를 쪼개나가는 방법이다. 예를 들어 "공부 시간이 10시간 이상"인지 아닌지로 데이터를 쪼갠다.

## 1. 정의

- P개의 feature($X_1,X_2,...,X_P$)로 구성된 공간을 J개의 겹치지 않는 구역($R_1,R_2,...,R_J$)으로 나눈다.
- $R_J$ 구역의 예측 값은 그 구역의 mean 값이다
- RSS를 최소화하는 구역을 찾는 것이 목표이고 RSS는 아래 수식과 같다.
    + 데이터들의 Y값과 영역 자체의 mean 값의 차이를 최소화하기

$$
\text{RSS} = \sum_{j=1}^J\sum_{i \in R_j}(Y^{(i)} - \hat{Y_{R_j}})^2
$$

## 2. 방법

- top-down, greedy approach -> recursive binary splitting